# Inference notebook

In [1]:
import numpy as np
import requests

In [2]:
# API_URL = "http://localhost:8080" # Uncomment for local testing
# Replace with Cloud Run URL from make get-api-url
API_URL = "https://rnn-api-ubrff53eoq-ew.a.run.app"

# Use config from model training
WINDOW_SIZE = 168

# Sample input data
PRICE = 6.4
LOAD = 83900.0
PRODUCTION = 78502.0

# Generate noisy sequence data
prices = [PRICE + np.random.normal(0, 0.5) for _ in range(WINDOW_SIZE)]
loads = [LOAD + np.random.normal(0, 1000) for _ in range(WINDOW_SIZE)]
productions = [PRODUCTION + np.random.normal(0, 1000) for _ in range(WINDOW_SIZE)]

payload = {"features": list(zip(prices, loads, productions))}

print(f"Generated payload with {len(payload['features'])} timesteps")

response = requests.post(f"{API_URL}/predict", json=payload)
response.raise_for_status()
prediction = response.json()["predicted_price"]

print(f"Predicted Price: {prediction}")

Generated payload with 168 timesteps
Predicted Price: 17.959152221679688
